In [1]:
#import basic requirements
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as utils
import numpy as np
dtype= torch.cuda.FloatTensor

In [2]:
def load_data(path='mnist.npz'):
    """Loads the MNIST dataset.
    # Arguments
        path: path where mnist is stored locallym, download from https://s3.amazonaws.com/img-datasets/mnist.npz
            
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)


In [3]:
def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]

    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    return categorical

In [4]:
#create dataset for training and testing
batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#print(x_train.shape, 'train samples')
#print(x_test.shape, 'test samples')
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)
ypred=[]


In [5]:
#Basic network definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 120)
        self.fc1_drop = nn.Dropout()
        self.fc2 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [6]:
#convert dataset and train
net=Net()
net=net.cuda()
#convert numpy arrays to torch tensors
Xtr=torch.stack([torch.Tensor(i) for i in x_train])
Ytr=torch.stack([torch.Tensor(i) for i in y_train])

Xte=torch.stack([torch.Tensor(i) for i in x_test])
Yte=torch.stack([torch.Tensor(i) for i in y_test])

traindataset=utils.TensorDataset(Xtr,Ytr)
traindataloader=utils.DataLoader(traindataset)


testdataset=utils.TensorDataset(Xte,Yte)
testdataloader=utils.DataLoader(testdataset)



epochs=100
optimizer=optim.SGD(net.parameters(),lr=0.01)
criterion=torch.nn.MSELoss()

In [7]:
def train():
    net.train()
    for batch,(data,target) in enumerate(traindataloader):
        data, target = data.cuda(), target.cuda()
        data,target=Variable(data),Variable(target)
        optimizer.zero_grad()
        #print data.size()
        output=net(data)
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch * len(data), len(traindataloader.dataset),
                100. * batch / len(traindataloader), loss.data[0]))


def test():
    net.eval()
    correct=0
    testloss=0
    
    #todo: must optimize bunch of functions
    for data,target in testdataloader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = net(data).cpu().data.numpy()[0]
        output=np.array(output)
        op=[]
        for i in output:
            if i==np.max(output):
                op.append(1)
            else:
                op.append(0)
        ypred.append(op)
        #print op



In [ ]:
#test for multiple epochs
for epoch in range(5):
    train()
    test()
ynewpr=[np.argmax(i,axis=-1) for i in ypred]
ynewte=[np.argmax(i,axis=-1) for i in y_test]
from sklearn.metrics import classification_report
print(classification_report(ynewte, ynewpr))

Train Epoch: 0 [0/60000 (0%)]	Loss: 5.867501
Train Epoch: 0 [10/60000 (0%)]	Loss: 5.871552
Train Epoch: 0 [20/60000 (0%)]	Loss: 5.863694
Train Epoch: 0 [30/60000 (0%)]	Loss: 5.871122
Train Epoch: 0 [40/60000 (0%)]	Loss: 5.858850
Train Epoch: 0 [50/60000 (0%)]	Loss: 5.865044
Train Epoch: 0 [60/60000 (0%)]	Loss: 5.863752
Train Epoch: 0 [70/60000 (0%)]	Loss: 5.866182
Train Epoch: 0 [80/60000 (0%)]	Loss: 5.868808
Train Epoch: 0 [90/60000 (0%)]	Loss: 5.865220
Train Epoch: 0 [100/60000 (0%)]	Loss: 5.870227
Train Epoch: 0 [110/60000 (0%)]	Loss: 5.865190
Train Epoch: 0 [120/60000 (0%)]	Loss: 5.864089
Train Epoch: 0 [130/60000 (0%)]	Loss: 5.863898
Train Epoch: 0 [140/60000 (0%)]	Loss: 5.865493
Train Epoch: 0 [150/60000 (0%)]	Loss: 5.870716
Train Epoch: 0 [160/60000 (0%)]	Loss: 5.865931
Train Epoch: 0 [170/60000 (0%)]	Loss: 5.862967
Train Epoch: 0 [180/60000 (0%)]	Loss: 5.856150
Train Epoch: 0 [190/60000 (0%)]	Loss: 5.865388
Train Epoch: 0 [200/60000 (0%)]	Loss: 5.858100
Train Epoch: 0 [210/6000

Train Epoch: 0 [1750/60000 (3%)]	Loss: 5.862836
Train Epoch: 0 [1760/60000 (3%)]	Loss: 5.863460
Train Epoch: 0 [1770/60000 (3%)]	Loss: 5.863679
Train Epoch: 0 [1780/60000 (3%)]	Loss: 5.863820
Train Epoch: 0 [1790/60000 (3%)]	Loss: 5.857522
Train Epoch: 0 [1800/60000 (3%)]	Loss: 5.863878
Train Epoch: 0 [1810/60000 (3%)]	Loss: 5.863338
Train Epoch: 0 [1820/60000 (3%)]	Loss: 5.863208
Train Epoch: 0 [1830/60000 (3%)]	Loss: 5.860955
Train Epoch: 0 [1840/60000 (3%)]	Loss: 5.850419
Train Epoch: 0 [1850/60000 (3%)]	Loss: 5.863588
Train Epoch: 0 [1860/60000 (3%)]	Loss: 5.857938
Train Epoch: 0 [1870/60000 (3%)]	Loss: 5.862707
Train Epoch: 0 [1880/60000 (3%)]	Loss: 5.866454
Train Epoch: 0 [1890/60000 (3%)]	Loss: 5.863032
Train Epoch: 0 [1900/60000 (3%)]	Loss: 5.864129
Train Epoch: 0 [1910/60000 (3%)]	Loss: 5.863119
Train Epoch: 0 [1920/60000 (3%)]	Loss: 5.862813
Train Epoch: 0 [1930/60000 (3%)]	Loss: 5.862430
Train Epoch: 0 [1940/60000 (3%)]	Loss: 5.862693
Train Epoch: 0 [1950/60000 (3%)]	Loss: 5

Train Epoch: 0 [3460/60000 (6%)]	Loss: 5.858986
Train Epoch: 0 [3470/60000 (6%)]	Loss: 5.865111
Train Epoch: 0 [3480/60000 (6%)]	Loss: 5.863634
Train Epoch: 0 [3490/60000 (6%)]	Loss: 5.863716
Train Epoch: 0 [3500/60000 (6%)]	Loss: 5.863445
Train Epoch: 0 [3510/60000 (6%)]	Loss: 5.863039
Train Epoch: 0 [3520/60000 (6%)]	Loss: 5.863540
Train Epoch: 0 [3530/60000 (6%)]	Loss: 5.864382
Train Epoch: 0 [3540/60000 (6%)]	Loss: 5.851308
Train Epoch: 0 [3550/60000 (6%)]	Loss: 5.858297
Train Epoch: 0 [3560/60000 (6%)]	Loss: 5.849530
Train Epoch: 0 [3570/60000 (6%)]	Loss: 5.864583
Train Epoch: 0 [3580/60000 (6%)]	Loss: 5.863233
Train Epoch: 0 [3590/60000 (6%)]	Loss: 5.852930
Train Epoch: 0 [3600/60000 (6%)]	Loss: 5.863670
Train Epoch: 0 [3610/60000 (6%)]	Loss: 5.865318
Train Epoch: 0 [3620/60000 (6%)]	Loss: 5.856896
Train Epoch: 0 [3630/60000 (6%)]	Loss: 5.864600
Train Epoch: 0 [3640/60000 (6%)]	Loss: 5.865860
Train Epoch: 0 [3650/60000 (6%)]	Loss: 5.862335
Train Epoch: 0 [3660/60000 (6%)]	Loss: 5

Train Epoch: 0 [5180/60000 (9%)]	Loss: 5.864303
Train Epoch: 0 [5190/60000 (9%)]	Loss: 5.863136
Train Epoch: 0 [5200/60000 (9%)]	Loss: 5.861845
Train Epoch: 0 [5210/60000 (9%)]	Loss: 5.867176
Train Epoch: 0 [5220/60000 (9%)]	Loss: 5.864768
Train Epoch: 0 [5230/60000 (9%)]	Loss: 5.850260
Train Epoch: 0 [5240/60000 (9%)]	Loss: 5.863420
Train Epoch: 0 [5250/60000 (9%)]	Loss: 5.862855
Train Epoch: 0 [5260/60000 (9%)]	Loss: 5.856755
Train Epoch: 0 [5270/60000 (9%)]	Loss: 5.855009
Train Epoch: 0 [5280/60000 (9%)]	Loss: 5.861438
Train Epoch: 0 [5290/60000 (9%)]	Loss: 5.854415
Train Epoch: 0 [5300/60000 (9%)]	Loss: 5.855575
Train Epoch: 0 [5310/60000 (9%)]	Loss: 5.854091
Train Epoch: 0 [5320/60000 (9%)]	Loss: 5.865007
Train Epoch: 0 [5330/60000 (9%)]	Loss: 5.846620
Train Epoch: 0 [5340/60000 (9%)]	Loss: 5.852276
Train Epoch: 0 [5350/60000 (9%)]	Loss: 5.865180
Train Epoch: 0 [5360/60000 (9%)]	Loss: 5.862830
Train Epoch: 0 [5370/60000 (9%)]	Loss: 5.864528
Train Epoch: 0 [5380/60000 (9%)]	Loss: 5

Train Epoch: 0 [6920/60000 (12%)]	Loss: 5.857800
Train Epoch: 0 [6930/60000 (12%)]	Loss: 5.861898
Train Epoch: 0 [6940/60000 (12%)]	Loss: 5.867799
Train Epoch: 0 [6950/60000 (12%)]	Loss: 5.863745
Train Epoch: 0 [6960/60000 (12%)]	Loss: 5.846643
Train Epoch: 0 [6970/60000 (12%)]	Loss: 5.863708
Train Epoch: 0 [6980/60000 (12%)]	Loss: 5.865005
Train Epoch: 0 [6990/60000 (12%)]	Loss: 5.852586
Train Epoch: 0 [7000/60000 (12%)]	Loss: 5.863678
Train Epoch: 0 [7010/60000 (12%)]	Loss: 5.862415
Train Epoch: 0 [7020/60000 (12%)]	Loss: 5.845926
Train Epoch: 0 [7030/60000 (12%)]	Loss: 5.859232
Train Epoch: 0 [7040/60000 (12%)]	Loss: 5.850554
Train Epoch: 0 [7050/60000 (12%)]	Loss: 5.849542
Train Epoch: 0 [7060/60000 (12%)]	Loss: 5.852147
Train Epoch: 0 [7070/60000 (12%)]	Loss: 5.871058
Train Epoch: 0 [7080/60000 (12%)]	Loss: 5.866727
Train Epoch: 0 [7090/60000 (12%)]	Loss: 5.853467
Train Epoch: 0 [7100/60000 (12%)]	Loss: 5.855164
Train Epoch: 0 [7110/60000 (12%)]	Loss: 5.864779
Train Epoch: 0 [7120

...

Train Epoch: 0 [23630/60000 (39%)]	Loss: 5.864465
Train Epoch: 0 [23640/60000 (39%)]	Loss: 5.865426
Train Epoch: 0 [23650/60000 (39%)]	Loss: 5.859007
Train Epoch: 0 [23660/60000 (39%)]	Loss: 5.847103
Train Epoch: 0 [23670/60000 (39%)]	Loss: 5.837545
Train Epoch: 0 [23680/60000 (39%)]	Loss: 5.863044
Train Epoch: 0 [23690/60000 (39%)]	Loss: 5.866307
Train Epoch: 0 [23700/60000 (40%)]	Loss: 5.851225
Train Epoch: 0 [23710/60000 (40%)]	Loss: 5.865289
Train Epoch: 0 [23720/60000 (40%)]	Loss: 5.842452
Train Epoch: 0 [23730/60000 (40%)]	Loss: 5.865739
Train Epoch: 0 [23740/60000 (40%)]	Loss: 5.844646
Train Epoch: 0 [23750/60000 (40%)]	Loss: 5.857660
Train Epoch: 0 [23760/60000 (40%)]	Loss: 5.843880
Train Epoch: 0 [23770/60000 (40%)]	Loss: 5.865874
Train Epoch: 0 [23780/60000 (40%)]	Loss: 5.852918
Train Epoch: 0 [23790/60000 (40%)]	Loss: 5.856749
Train Epoch: 0 [23800/60000 (40%)]	Loss: 5.862415
Train Epoch: 0 [23810/60000 (40%)]	Loss: 5.843793
Train Epoch: 0 [23820/60000 (40%)]	Loss: 5.850471


Train Epoch: 0 [25280/60000 (42%)]	Loss: 5.866501
Train Epoch: 0 [25290/60000 (42%)]	Loss: 5.848630
Train Epoch: 0 [25300/60000 (42%)]	Loss: 5.850739
Train Epoch: 0 [25310/60000 (42%)]	Loss: 5.851254
Train Epoch: 0 [25320/60000 (42%)]	Loss: 5.854080
Train Epoch: 0 [25330/60000 (42%)]	Loss: 5.863662
Train Epoch: 0 [25340/60000 (42%)]	Loss: 5.839475
Train Epoch: 0 [25350/60000 (42%)]	Loss: 5.844884
Train Epoch: 0 [25360/60000 (42%)]	Loss: 5.839389
Train Epoch: 0 [25370/60000 (42%)]	Loss: 5.842029
Train Epoch: 0 [25380/60000 (42%)]	Loss: 5.840381
Train Epoch: 0 [25390/60000 (42%)]	Loss: 5.844743
Train Epoch: 0 [25400/60000 (42%)]	Loss: 5.841794
Train Epoch: 0 [25410/60000 (42%)]	Loss: 5.846041
Train Epoch: 0 [25420/60000 (42%)]	Loss: 5.837481
Train Epoch: 0 [25430/60000 (42%)]	Loss: 5.861139
Train Epoch: 0 [25440/60000 (42%)]	Loss: 5.866924
Train Epoch: 0 [25450/60000 (42%)]	Loss: 5.842380
Train Epoch: 0 [25460/60000 (42%)]	Loss: 5.838442
Train Epoch: 0 [25470/60000 (42%)]	Loss: 5.859471


Train Epoch: 0 [26970/60000 (45%)]	Loss: 5.863221
Train Epoch: 0 [26980/60000 (45%)]	Loss: 5.837672
Train Epoch: 0 [26990/60000 (45%)]	Loss: 5.848256
Train Epoch: 0 [27000/60000 (45%)]	Loss: 5.862803
Train Epoch: 0 [27010/60000 (45%)]	Loss: 5.855151
Train Epoch: 0 [27020/60000 (45%)]	Loss: 5.851497
Train Epoch: 0 [27030/60000 (45%)]	Loss: 5.855348
Train Epoch: 0 [27040/60000 (45%)]	Loss: 5.843302
Train Epoch: 0 [27050/60000 (45%)]	Loss: 5.857929
Train Epoch: 0 [27060/60000 (45%)]	Loss: 5.851265
Train Epoch: 0 [27070/60000 (45%)]	Loss: 5.862491
Train Epoch: 0 [27080/60000 (45%)]	Loss: 5.856456
Train Epoch: 0 [27090/60000 (45%)]	Loss: 5.845095
Train Epoch: 0 [27100/60000 (45%)]	Loss: 5.872056
Train Epoch: 0 [27110/60000 (45%)]	Loss: 5.864019
Train Epoch: 0 [27120/60000 (45%)]	Loss: 5.863216
Train Epoch: 0 [27130/60000 (45%)]	Loss: 5.870687
Train Epoch: 0 [27140/60000 (45%)]	Loss: 5.840509
Train Epoch: 0 [27150/60000 (45%)]	Loss: 5.854819
Train Epoch: 0 [27160/60000 (45%)]	Loss: 5.851281


Train Epoch: 0 [28630/60000 (48%)]	Loss: 5.861215
Train Epoch: 0 [28640/60000 (48%)]	Loss: 5.872413
Train Epoch: 0 [28650/60000 (48%)]	Loss: 5.846671
Train Epoch: 0 [28660/60000 (48%)]	Loss: 5.839968
Train Epoch: 0 [28670/60000 (48%)]	Loss: 5.848546
Train Epoch: 0 [28680/60000 (48%)]	Loss: 5.845024
Train Epoch: 0 [28690/60000 (48%)]	Loss: 5.839044
Train Epoch: 0 [28700/60000 (48%)]	Loss: 5.854741
Train Epoch: 0 [28710/60000 (48%)]	Loss: 5.867171
Train Epoch: 0 [28720/60000 (48%)]	Loss: 5.872058
Train Epoch: 0 [28730/60000 (48%)]	Loss: 5.843771
Train Epoch: 0 [28740/60000 (48%)]	Loss: 5.855889
Train Epoch: 0 [28750/60000 (48%)]	Loss: 5.865204
Train Epoch: 0 [28760/60000 (48%)]	Loss: 5.846006
Train Epoch: 0 [28770/60000 (48%)]	Loss: 5.871823
Train Epoch: 0 [28780/60000 (48%)]	Loss: 5.846814
Train Epoch: 0 [28790/60000 (48%)]	Loss: 5.838292
Train Epoch: 0 [28800/60000 (48%)]	Loss: 5.844161
Train Epoch: 0 [28810/60000 (48%)]	Loss: 5.864628
Train Epoch: 0 [28820/60000 (48%)]	Loss: 5.844750


Train Epoch: 0 [30280/60000 (50%)]	Loss: 5.865077
Train Epoch: 0 [30290/60000 (50%)]	Loss: 5.858727
Train Epoch: 0 [30300/60000 (50%)]	Loss: 5.838296
Train Epoch: 0 [30310/60000 (51%)]	Loss: 5.841205
Train Epoch: 0 [30320/60000 (51%)]	Loss: 5.837390
Train Epoch: 0 [30330/60000 (51%)]	Loss: 5.857005
Train Epoch: 0 [30340/60000 (51%)]	Loss: 5.844532
Train Epoch: 0 [30350/60000 (51%)]	Loss: 5.843630
Train Epoch: 0 [30360/60000 (51%)]	Loss: 5.865026
Train Epoch: 0 [30370/60000 (51%)]	Loss: 5.837772
Train Epoch: 0 [30380/60000 (51%)]	Loss: 5.844504
Train Epoch: 0 [30390/60000 (51%)]	Loss: 5.876076
Train Epoch: 0 [30400/60000 (51%)]	Loss: 5.849883
Train Epoch: 0 [30410/60000 (51%)]	Loss: 5.869652
Train Epoch: 0 [30420/60000 (51%)]	Loss: 5.862080
Train Epoch: 0 [30430/60000 (51%)]	Loss: 5.846749
Train Epoch: 0 [30440/60000 (51%)]	Loss: 5.841483
Train Epoch: 0 [30450/60000 (51%)]	Loss: 5.862415
Train Epoch: 0 [30460/60000 (51%)]	Loss: 5.845536
Train Epoch: 0 [30470/60000 (51%)]	Loss: 5.867388
